In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
GOOGLE_APPLICATION_CREDENTIALS = "/Users/tararosen/Downloads/hacker-news-79073e57c2a7.json"

In [ ]:
# import our bq_helper package
import bq_helper

In [ ]:
# create a helper object for our bigquery dataset
hacker_news = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "hacker_news")

In [ ]:
# print a list of all the tables in the hacker_news dataset
hacker_news.list_tables()

In [ ]:
# print information on all the columns in the "full" table
# in the hacker_news dataset
hacker_news.table_schema("full")

In [ ]:
# preview the first couple lines of the "full" table
hacker_news.head("full")

In [ ]:
# preview the first ten entries in the by column of the full table
hacker_news.head("full", selected_columns="by", num_rows=10)

In [ ]:
# this query looks in the full table in the hacker_news
# dataset, then gets the score column from every row where 
# the type column has "job" in it.
query = """SELECT score
            FROM `bigquery-public-data.hacker_news.full`
            WHERE type = "job" """

# check how big this query will be
hacker_news.estimate_query_size(query)

Important: When you're writing your query, make sure that the name of the table (next to FROM) is in back ticks (`), not single quotes ('). The reason for this is that the names of BigQuery tables contain periods in them, which in SQL are special characters. Putting the table name in back ticks protects the table name, so it's treated as a single string instead of being run as code.

In [ ]:
# only run this query if it's less than 100 MB
hacker_news.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
# check out the scores of job postings (if the 
# query is smaller than 1 gig)
job_post_scores = hacker_news.query_to_pandas_safe(query)

In [ ]:
job_post_scores.head()

In [ ]:
# average score for job posts
job_post_scores['score'].mean()

In [ ]:
job_post_scores.score.mean()

In [ ]:
# save our dataframe as a .csv 
job_post_scores.to_csv("job_post_scores.csv")

### SQL Scavenger Hunt Day 1 - SELECT, FROM and WHERE

In [ ]:
# create a helper object for this dataset
open_aq = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="openaq")

# print all the tables in this dataset (there's only one!)
open_aq.list_tables()

In [ ]:
open_aq.head('global_air_quality')

In [ ]:
query = """SELECT city
           FROM `bigquery-public-data.openaq.global_air_quality`
           WHERE country = 'US'"""

In [ ]:
us_cities = open_aq.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
us_cities

In [ ]:
us_cities['city'].value_counts().head()

Here's the questions I would like you to get the data to answer:

* Which countries use a unit other than ppm to measure any type of pollution? (Hint: to get rows where the value isn't something, use "!=")
* Which pollutants have a value of exactly 0?

In [ ]:
# find column names
open_aq.table_schema('global_air_quality')

In [ ]:
# Which countries use a unit other than ppm to measure any type of pollution?
query = ("""SELECT country
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE pollutant != 'ppm'""")

In [ ]:
not_ppm = open_aq.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
not_ppm['country'].value_counts()

In [ ]:
not_ppm['country'].unique()

In [ ]:
# Which pollutants have a value of exactly 0?

query = ("""SELECT DISTINCT(pollutant)
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE value = 0 """)

In [ ]:
ppm_zero = open_aq.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
ppm_zero

### SQL Scavenger Hunt Day 2 - GROUP BY... HAVING and COUNT

In [ ]:
# create a helper object for our bigquery dataset
hacker_news = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "hacker_news")

In [ ]:
hacker_news.head('comments')

In [ ]:
query = """SELECT parent, COUNT(id)
           FROM `bigquery-public-data.hacker_news.comments`
           GROUP BY parent
           HAVING COUNT(id) > 200"""

In [ ]:
responses = hacker_news.query_to_pandas_safe(query, max_gb_scanned=0.1)

Here's the questions I would like you to get the data to answer:

* How many stories (use the "id" column) are there of each type (in the "type" column) in the full table?
* How many comments have been deleted? (If a comment was deleted the "deleted" column in the comments table will have the value "True".)
* Optional extra credit: read about aggregate functions other than COUNT() and modify one of the queries you wrote above to use a different aggregate function.

In [ ]:
# print a list of all the tables in the hacker_news dataset
hacker_news.list_tables()

In [ ]:
hacker_news.head('full')

In [ ]:
hacker_news['full']

In [ ]:
# How many stories (use the "id" column) are there of each type (in the "type" column) in the full table?

query = """SELECT type, COUNT(id)
           FROM `bigquery-public-data.hacker_news.full`
           GROUP BY type
        """

In [ ]:
stories_by_type = hacker_news.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
# How many comments have been deleted? (If a comment was deleted the "deleted" column in the comments table will have the value "True".)

query = """SELECT deleted, COUNT(id)
           FROM `bigquery-public-data.hacker_news.full`
           WHERE deleted = True
           GROUP BY deleted
           """

In [ ]:
deleted = hacker_news.query_to_pandas_safe(query, max_gb_scanned=0.1)

### SQL Scavenger Hunt Day 3 - ORDER BY and DATE

In [ ]:
# create a helper object for this dataset
accidents = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="nhtsa_traffic_fatalities")

In [ ]:
# print a list of all the tables in the accidents dataset
accidents.list_tables()

In [ ]:
# print information on all the columns in the "accident_2015" table
# in the accidents dataset
accidents.table_schema("accident_2015")

In [ ]:
# we're going to look at which day of the week the most fatal traffic accidents happen on
# in 2015

query = """SELECT COUNT(consecutive_number),
           EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
           FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
           GROUP BY EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
           ORDER BY COUNT(consecutive_number) DESC
        """

In [ ]:
day_most_fatal_2015 = accidents.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
day_most_fatal_2015

* 1 - Sunday
* 2 - Monday
* 3 - Tuesday
* 4 - Wednesday
* 5 - Thursday
* 6 - Friday
* 7 - Saturday

The most fatal accidents in 2015 occurred on Saturday and the fewest occurred on Tuesday.

In [ ]:
# library for plotting
import matplotlib.pyplot as plt

# make a plot to show that our data is, actually, sorted:
plt.plot(day_most_fatal_2015.f0_)
plt.title("Number of Accidents by Rank of Day \n (Most to least dangerous)")

In [ ]:
# print information on all the columns in the "accident_2016" table
# in the accidents dataset
accidents.table_schema("accident_2016")

In [ ]:
# we're going to look at which day of the week the most fatal traffic accidents happen on in 2016

query = """SELECT COUNT(consecutive_number),
           EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
           FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
           GROUP BY EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
           ORDER BY COUNT(consecutive_number) DESC"""

In [ ]:
day_most_fatal_2016 = accidents.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
day_most_fatal_2016

The most fatal accidents in 2016 occurred on Saturday and the fewest occurred on Tuesday.

In [ ]:
# library for plotting
import matplotlib.pyplot as plt

# make a plot to show that our data is, actually, sorted:
plt.plot(day_most_fatal_2016.f0_)
plt.title("Number of Accidents by Rank of Day \n (Most to least dangerous)")

Here are the questions I would like you to get the data to answer:

* Which hours of the day do the most accidents occur during? <br/>
<br/>
** Return a table that has information on how many accidents occurred in each hour of the day in 2015, sorted by the the number of accidents which occurred each hour. Use either the accident_2015 or accident_2016 table for this, and the timestamp_of_crash column. (Yes, there is an hour_of_crash column, but if you use that one you won't get a chance to practice with dates. :P)
Hint: You will probably want to use the EXTRACT() function for this. **

* Which state has the most hit and runs? <br/>
<br/>
** Return a table with the number of vehicles registered in each state that were involved in hit-and-run accidents, sorted by the number of hit and runs. Use either the vehicle_2015 or vehicle_2016 table for this, especially the registration_state_name and hit_and_run columns. **

In [ ]:
# Which hours of the day do the most accidents occur during?

query = """SELECT COUNT(consecutive_number),
                  EXTRACT(HOUR FROM timestamp_of_crash)
           FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
           GROUP BY EXTRACT(HOUR FROM timestamp_of_crash)
           ORDER BY 1 DESC
        """

In [ ]:
by_hours_2015 = accidents.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
by_hours_2015

The most fatal accidents occur during the 6 pm hour. The least during the 4 AM hour.

In [ ]:
accidents.head('vehicle_2015')

In [ ]:
# Which state has the most hit and runs?
# registration_state_name and hit_and_run columns


query = """SELECT COUNT(hit_and_run),
                  registration_state_name
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.vehicle_2015`
            WHERE hit_and_run = 'Yes'
            GROUP BY registration_state_name
            ORDER BY 1 DESC                  
"""

In [ ]:
hit_run_by_state = accidents.query_to_pandas_safe(query, max_gb_scanned=0.1)

In [ ]:
hit_run_by_state